# Azure AI Search 셋업

이 노트북에서는 Azure AI Search를 활용을 위한 인덱스를 생성합니다.

## 0. 필요한 라이브러리 설치

In [1]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install python-dotenv azure-identity

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os, shutil

AZ_DIR = "/opt/homebrew/bin"
os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]

print("az which:", shutil.which("az"))

az which: /opt/homebrew/bin/az


## 1. 변수 등록 

In [ ]:
##############################################
# 1. 변수 등록
##############################################

# Azure AI Search 정보
AZURE_SEARCH_SERVICE_ENDPOINT="<your_search_service_endpoint>"
AZURE_SEARCH_INDEX_NAME="<your_search_index_name>"

# Verctor 임베딩 차원
EMBEDDING_DIMENSION = 3072 #Embedding model 과 동일 차원으로 설정

In [10]:
import os
from dotenv import load_dotenv

##############################################
# 2. 변수 확인
##############################################

# .env 파일에서 환경 변수 로드
load_dotenv()

##############################################
# 3. 환경 변수 설정 확인
##############################################
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", AZURE_SEARCH_SERVICE_ENDPOINT)
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME", AZURE_SEARCH_INDEX_NAME)

print(f"Search Service Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index Name: {AZURE_SEARCH_INDEX_NAME}")
print(f"Embedding dimension:", EMBEDDING_DIMENSION)

Search Service Endpoint: https://ai-search-commercial-1023.search.windows.net
Search Index Name: products-index
Embedding dimension: 3072


## 2. 인덱스 생성

In [ ]:
from azure.core.exceptions import HttpResponseError
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes import SearchIndexClient
import os, shutil

##############################################
# 2-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))

##############################################
# 2-1. 인덱스 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    credential=credential
)

##############################################
# 2-2. 기존 인덱스 존재 여부 확인 및 삭제
##############################################
# 기존 인덱스가 있으면 삭제합니다.
try:
    index_client.get_index(AZURE_SEARCH_INDEX_NAME)
    print(f"기존 인덱스 '{AZURE_SEARCH_INDEX_NAME}'을(를) 발견했습니다. 삭제를 시도합니다...")

    # 기존 인덱스 삭제
    index_client.delete_index(AZURE_SEARCH_INDEX_NAME)
    print(f"기존 인덱스 '{AZURE_SEARCH_INDEX_NAME}' 삭제 완료.")
except HttpResponseError as e:
    if e.status_code == 404:
        # 404 Not Found → 인덱스가 없으므로 새로 생성 진행
        print(f"기존 인덱스 '{AZURE_SEARCH_INDEX_NAME}'이(가) 없습니다. 새로 생성을 시작합니다.")
    else:
        print(f"인덱스 확인 중 오류 발생: {e}")
        raise
except Exception as e:
    print(f"예상치 못한 오류 발생 (인덱스 확인/삭제 중): {e}")
    raise


#############################################

기존 인덱스 'products-index'을(를) 발견했습니다. 삭제를 시도합니다...
기존 인덱스 'products-index' 삭제 완료.


In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
)


##############################################
# 2-3. 인덱스 스키마 정의
# : 기본 아이디, 상품명, 브랜드, 설명, 이미지 URL, 가격, 설명 벡터 필드
##############################################
fields = [
    
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
    SimpleField(name="price", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
    SearchableField(name="name", type=SearchFieldDataType.String, searchable=True, analyzer_name="ko.microsoft"),
    SearchableField(name="brand", type=SearchFieldDataType.String, searchable=True, analyzer_name="ko.microsoft"),
    SearchableField(name="description", type=SearchFieldDataType.String, searchable=True, analyzer_name="ko.microsoft"),
    
    SimpleField(name="imageUrl", type=SearchFieldDataType.String, filterable=False),
    SearchableField(name="imageCaption", type=SearchFieldDataType.String, searchable=True, analyzer_name="ko.microsoft"),
    SearchableField(name="imageDescription", type=SearchFieldDataType.String, searchable=True, analyzer_name="ko.microsoft"),
    SearchField(name="imageTags", type=SearchFieldDataType.Collection(SearchFieldDataType.String), searchable=True),
    

    # 벡터 임베딩 필드 (HNSW 인덱스 적용)
    SearchField(
        name="descriptionVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        hidden=False,
        vector_search_dimensions=EMBEDDING_DIMENSION,
        vector_search_profile_name="rag-hnsw-profile",
    ),
]

##############################################
# 2-4. 벡터 검색 구성 (HNSW 알고리즘)
##############################################
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="rag-hnsw-profile",
            algorithm_configuration_name="rag-hnsw-config"
        )
    ],
    algorithms=[
        HnswAlgorithmConfiguration(
            name="rag-hnsw-config",
            kind="hnsw",
            parameters={"metric": "cosine"}
        )
    ],
)

##############################################
# 2-5. 인덱스 객체 생성 및 생성 요청
##############################################
index = SearchIndex(
    name=AZURE_SEARCH_INDEX_NAME,
    fields=fields,
    vector_search=vector_search,
)

try:
    result = index_client.create_or_update_index(index)
    print(f"✅ 이커머스 인덱스 생성 완료: {result.name}")
except Exception as e:
    print(f"❌ 인덱스 생성 실패: {e}")
    raise

## 3. 인덱스 정보 확인

In [ ]:
##############################################
# 3. 생성된 인덱스 정보 조회
##############################################

# 인덱스 정보 가져오기
index_info = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

print(f"\n{'='*60}")
print(f"📋 인덱스 정보")
print(f"{'='*60}")
print(f"인덱스 이름: {index_info.name}")
print(f"총 필드 수: {len(index_info.fields)}개")
print(f"\n필드 목록:")
for field in index_info.fields:
    field_type = field.type
    properties = []
    if getattr(field, 'key', False):
        properties.append("Key")
    if getattr(field, 'searchable', False):
        properties.append("Searchable")
    if getattr(field, 'filterable', False):
        properties.append("Filterable")
    if getattr(field, 'sortable', False):
        properties.append("Sortable")
    if getattr(field, 'facetable', False):
        properties.append("Facetable")
    
    props_str = f" [{', '.join(properties)}]" if properties else ""
    
    # Analyzer 정보 추가
    analyzer_info = ""
    analyzer = getattr(field, 'analyzer', None) or getattr(field, 'analyzer_name', None)
    if analyzer:
        analyzer_info = f" | Analyzer: {analyzer}"
    
    print(f"  - {field.name}: {field_type}{props_str}{analyzer_info}")

if index_info.vector_search:
    print(f"\n벡터 검색 프로필:")
    for profile in index_info.vector_search.profiles:
        print(f"  - {profile.name}")

if hasattr(index_info, 'semantic_search') and index_info.semantic_search:
    print(f"\n시맨틱 검색 구성:")
    for config in index_info.semantic_search.configurations:
        print(f"  - {config.name}")

print(f"{'='*60}\n")